# Recommender Systems Pipeline - Рекомендательные системы

Пайплайны для:
- Collaborative Filtering (ALS, SVD)
- Content-Based Filtering
- Hybrid подходы
- Neural Collaborative Filtering

In [ ]:
!pip install surprise scikit-learn pandas numpy torch -q

In [ ]:
import pandas as pd
import numpy as np
from surprise import SVD, KNNBasic, Dataset, Reader
from surprise.model_selection import cross_validate, train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
warnings.filterwarnings('ignore')

print("✓ Библиотеки загружены!")

## 1. Загрузка данных

In [ ]:
# === ВАШИ ДАННЫЕ ===
# Формат: user_id, item_id, rating
ratings_df = pd.read_csv('ratings.csv')
# items_df = pd.read_csv('items.csv')  # Метаданные о товарах/фильмах

print(f"Ratings shape: {ratings_df.shape}")
print(f"\nПример данных:\n{ratings_df.head()}")
print(f"\nСтатистика:\n{ratings_df.describe()}")

## 2. Collaborative Filtering (SVD)

In [ ]:
# Подготовка данных для Surprise
reader = Reader(rating_scale=(1, 5))  # Укажите свою шкалу рейтингов
data = Dataset.load_from_df(ratings_df[['user_id', 'item_id', 'rating']], reader)

# Разделение на train/test
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# SVD модель
svd_model = SVD(
    n_factors=100,
    n_epochs=20,
    lr_all=0.005,
    reg_all=0.02
)

# Обучение
svd_model.fit(trainset)

# Оценка
from surprise import accuracy
predictions = svd_model.test(testset)
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)

print(f"\nRMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")

In [ ]:
# Получение рекомендаций для пользователя
def get_top_n_recommendations(model, user_id, n=10, ratings_df=None):
    """Получить топ-N рекомендаций для пользователя"""
    
    # Все item_id
    all_items = ratings_df['item_id'].unique()
    
    # Товары, которые пользователь уже оценил
    rated_items = ratings_df[ratings_df['user_id'] == user_id]['item_id'].values
    
    # Товары для предсказания
    items_to_predict = [item for item in all_items if item not in rated_items]
    
    # Предсказания
    predictions = [model.predict(user_id, item) for item in items_to_predict]
    
    # Сортировка по рейтингу
    predictions.sort(key=lambda x: x.est, reverse=True)
    
    return [(pred.iid, pred.est) for pred in predictions[:n]]

# Пример
user_id = 1
recommendations = get_top_n_recommendations(svd_model, user_id, n=10, ratings_df=ratings_df)
print(f"\nТоп-10 рекомендаций для пользователя {user_id}:")
for item_id, rating in recommendations:
    print(f"Item {item_id}: predicted rating {rating:.2f}")

## 3. Content-Based Filtering

In [ ]:
# Пример с текстовыми признаками товаров
# items_df должен содержать колонку с описанием или жанрами

# items_df = pd.read_csv('items.csv')
# # Колонка 'description' или 'genres'

# # TF-IDF векторизация
# tfidf = TfidfVectorizer(max_features=5000, stop_words='english')
# tfidf_matrix = tfidf.fit_transform(items_df['description'])

# # Косинусное сходство
# cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# def get_content_recommendations(item_id, n=10):
#     """Рекомендации на основе схожести контента"""
#     idx = items_df[items_df['item_id'] == item_id].index[0]
#     sim_scores = list(enumerate(cosine_sim[idx]))
#     sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
#     sim_scores = sim_scores[1:n+1]
#     item_indices = [i[0] for i in sim_scores]
#     return items_df.iloc[item_indices]['item_id'].values

# # Пример
# similar_items = get_content_recommendations(item_id=100, n=10)
# print(f"Похожие товары: {similar_items}")

## 4. Создание предсказаний для соревнования

In [ ]:
# === ТЕСТОВЫЕ ДАННЫЕ ===
# test_df должен содержать пары user_id, item_id для предсказания
test_df = pd.read_csv('test.csv')

# Предсказания
predictions = []
for _, row in test_df.iterrows():
    pred = svd_model.predict(row['user_id'], row['item_id'])
    predictions.append(pred.est)

# Submission
submission = pd.DataFrame({
    'id': test_df.index if 'id' not in test_df.columns else test_df['id'],
    'prediction': predictions
})

submission.to_csv('recsys_submission.csv', index=False)
print("✓ Submission сохранен!")

## 5. KNN Collaborative Filtering

In [ ]:
# User-based CF
sim_options = {
    'name': 'cosine',
    'user_based': True
}
knn_model = KNNBasic(sim_options=sim_options)
knn_model.fit(trainset)

# Оценка
knn_predictions = knn_model.test(testset)
knn_rmse = accuracy.rmse(knn_predictions)
print(f"KNN RMSE: {knn_rmse:.4f}")

## 6. Гибридный подход

In [ ]:
# Комбинирование CF и Content-Based
# def hybrid_recommendations(user_id, item_id, alpha=0.7):
#     """alpha - вес для CF, (1-alpha) - вес для Content-Based"""
#     cf_pred = svd_model.predict(user_id, item_id).est
#     # content_score = ...  # Получить из content-based
#     # hybrid_score = alpha * cf_pred + (1 - alpha) * content_score
#     return cf_pred